In [ ]:
# !pip install ragas langchain langchain-openai

In [49]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from langchain_text_splitters import TokenTextSplitter
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough 
from langchain_core.output_parsers import StrOutputParser
from langchain_core.documents import Document
from ragas import evaluate
from ragas.testset import TestsetGenerator
from ragas.metrics import LLMContextRecall, LLMContextPrecisionWithReference, Faithfulness, AnswerRelevancy
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

from src.utils.config import ConfigDB
from src.agent.prompts.analyst_prompt import ANALYSIS_SYSTEM_PROMPT




/var/folders/by/sbz15vxn1xz8499rjvlm_ctm0000gn/T/ipykernel_57529/441593575.py:11: DeprecationWarning: Importing LLMContextRecall from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import LLMContextRecall
  from ragas.metrics import LLMContextRecall, LLMContextPrecisionWithReference, Faithfulness, AnswerRelevancy
/var/folders/by/sbz15vxn1xz8499rjvlm_ctm0000gn/T/ipykernel_57529/441593575.py:11: DeprecationWarning: Importing LLMContextPrecisionWithReference from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import LLMContextPrecisionWithReference
  from ragas.metrics import LLMContextRecall, LLMContextPrecisionWithReference, Faithfulness, AnswerRelevancy
/var/folders/by/sbz15vxn1xz8499rjvlm_ctm0000gn/T/ipykernel_57529/441593575.py:11: DeprecationWarning: Importing Faithfulness from 'ragas.m

In [44]:
COLLECTION_NAME = ConfigDB.COLLECTION_NAME
EMBEDDING_MODEL = ConfigDB.EMBEDDING_MODEL
VECTOR_SIZE_MAP = ConfigDB.VECTOR_SIZE_MAP[EMBEDDING_MODEL]
OPENAI_MODEL = "gpt-4o-mini"

COLLECTION_NAME, EMBEDDING_MODEL, VECTOR_SIZE_MAP, OPENAI_MODEL

('learning_ai', 'text-embedding-3-large', 3072, 'gpt-4o-mini')

In [25]:
# 데이터셋을 생성할 때 사용할 context를 추출 - sampling
client = QdrantClient(host=ConfigDB.HOST, port=ConfigDB.PORT)

# 전체 데이터를 다 조회해서 그 중 랜덤하게 K개만 sampling
info = client.get_collection(COLLECTION_NAME)
total_count = info.points_count

print("info : ", info)
print("total_count : ", total_count)

info :  status=<CollectionStatus.GREEN: 'green'> optimizer_status=<OptimizersStatusOneOf.OK: 'ok'> warnings=None indexed_vectors_count=15315 points_count=15315 segments_count=4 config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=3072, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None, inline_storage=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=10000, flush_interval_sec=5, max_optimization_threads=None), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahe

In [27]:
results, next_id = client.scroll(
    collection_name=COLLECTION_NAME,
    limit=info.points_count,
)

# sampling
import random
sample_dataset = random.sample(results, 5) # 리스트에서 랜덤하게 K개를 추출


# 문서 내용만 추출
docs = [point.payload['page_content'] for point in sample_dataset]
docs

["[KEYWORDS] signals\n[TITLE] signal\n[H1] signal --- Set handlers for asynchronous events\n[H2] Module contents\n\ndata SIGUSR2\n\n   User-defined signal 2.\n\ndata SIGVTALRM\n\n   Virtual timer expired.\n\ndata SIGWINCH\n\n   Window resize signal.\n\ndata SIGXCPU\n\n   CPU time limit exceeded.\n\ndata SIG*\n\n   All the signal numbers are defined symbolically.  For example, the hangup signal\n   is defined as signal.SIGHUP; the variable names are identical to the\n   names used in C programs, as found in ``<signal.h>``.  The Unix man page for\n   '``signal``' lists the existing signals (on some systems this is\n   not all systems define the same set of signal names; only those names defined by\n   the system are defined by this module.\n\ndata CTRL_C_EVENT\n\n   The signal corresponding to the Ctrl+C keystroke event. This signal can\n   only be used with os.kill.\n\ndata CTRL_BREAK_EVENT\n\n   The signal corresponding to the Ctrl+Break keystroke event. This signal can\n   only be use

In [28]:
#################################################################
# Vector DB 연결
# retriever 생성
#################################################################

def get_vectorstore(collection_name: str):
    # 1. 임베딩 모델 설정
    dense_embeddings = OpenAIEmbeddings(model=EMBEDDING_MODEL)
    # 2. Qdrant 클라이언트 연결
    client = QdrantClient(host="localhost", port=6333)
    # 3. 기존 컬렉션에 연결
    vectorstore = QdrantVectorStore(
        client=client,
        collection_name=collection_name,
        embedding=dense_embeddings
    )
    
    return vectorstore


def get_retriever(vectorstore, k: int = 5):
    retriever = vectorstore.as_retriever(
        search_kwargs={"k": k}
    )
    return retriever

In [ ]:
vectorstore = get_vectorstore(COLLECTION_NAME)
retriever = get_retriever(vectorstore)

In [70]:
################################################################################
# 평가할 RAG Chain
################################################################################
prompt_txt = ANALYSIS_SYSTEM_PROMPT
prompt = ChatPromptTemplate.from_template(
    template=prompt_txt
)


def format_docs(documents:list)->str:
    """
    VectorStore에 조회한 문서들에서 내용(page_content)만 추출해서 str으로 합쳐서 반환.
    VectorStore의 검색결과인 List[Document]를 받아서 Document들에서 page_content의 내용만 추출한다.
    
    Args:
        documents(list[Document]): [Document(..), Document(...), ..]}
    Returns:
        str: 각 문서의 내용을 "\n\n"으로 연결한 string
    """
    return "\n\n".join(doc.page_content for doc in documents)

model = ChatOpenAI(model=OPENAI_MODEL)
parser = StrOutputParser()

# RAG 평가를 위해서 "답변", "검색한 문서" 둘이 출력되도록 변경.
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from operator import itemgetter

def format_doc_list(docs_dict: dict) -> list:
    # dictionary[list[Document], query:str] -> list[str]  
    # 문서내용만 추출해서(Document.page_content)만 추출한 리스트
    return [doc.page_content for doc in docs_dict['context']]

# dict | dict -> dict
# RunnablePassthrough() | dict | dict ==> RunnableSequence
chain = RunnablePassthrough() | {
    "context":retriever,
    "query":RunnablePassthrough()
} | {
    "response": prompt | model | parser,
    "retrieved_context": format_doc_list, # RAGAS 평가시 context -> List[str]
}

In [71]:
generator_llm = LangchainLLMWrapper(ChatOpenAI(model=OPENAI_MODEL))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model=EMBEDDING_MODEL))

generator = TestsetGenerator(
    llm=generator_llm,
    embedding_model=generator_embeddings,
    llm_context="데이터셋은 반드시 한국어로 작성한다. 데이터셋은 JSON 문법을 꼭 지켜서 작성한다.\
특히 구두점은 꼭 지켜야 한다. Document에 JSON 문법에 맞지 않는 표현이 있으면 반드시 수정해서 처리한다."  
    # 질문/답변 생성할 때 llm에 전달할 추가 system prompt를 설정.
)

# testset
testset = generator.generate_with_chunks(
    docs, testset_size=10  # context 내용, 테스트데이터셋 몇개를 만들지.
)

/var/folders/by/sbz15vxn1xz8499rjvlm_ctm0000gn/T/ipykernel_57529/4276533792.py:1: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  generator_llm = LangchainLLMWrapper(ChatOpenAI(model=OPENAI_MODEL))
/var/folders/by/sbz15vxn1xz8499rjvlm_ctm0000gn/T/ipykernel_57529/4276533792.py:2: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model=EMBEDDING_MODEL))
Applying SummaryExtractor:   0%|          | 0/5 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/5 [00:00<?, ?it/s]Node 6aba899f-b384-416d-8c03-9ce95c61c858 does not have a summary. Skipping filtering.
Node 3b0fa0bc-114b-4cca-af87-67d77b1a5d88 does not have a summary. Skipping filtering.
Node 4c597e6a-1114-4f29-92c7-4d47f81abc5f does not have a summary. Skipping filtering.
Node fa296c6a-65fb-4bcb-95c5-0c0f9318fffd does not have a summary. Skipping filtering.
Node bd7cb3a2-6259-492c-a377-e5d02b857950 does not have a summary. Skipping filtering.
Applying OverlapScoreBuilder: 100%|██████████| 1/1 [00:00<00:00, 1117.59it/s]
Skipping multi_hop_abstract_query_synthesizer due to unexpected error: No relationships match the provided condition. Cannot form clusters.
Generating Samples: 100%|██████████| 8/8 [00:09<00:00,  1.13s/it]


In [72]:
eval_df = testset.to_pandas()
eval_df.head()

,user_input,reference_contexts,reference,persona_name,query_style,query_length,synthesizer_name
0,What does SIGUSR2 represent in signal handling?,[[KEYWORDS] signals\n[TITLE] signal\n[H1] sign...,SIGUSR2 is defined as User-defined signal 2.,Email Automation Developer,WEB_SEARCH_LIKE,SHORT,single_hop_specific_query_synthesizer
1,What does SIGWINCH represent in signal handling?,[[KEYWORDS] signals\n[TITLE] signal\n[H1] sign...,SIGWINCH represents the window resize signal.,Python Software Developer,PERFECT_GRAMMAR,SHORT,single_hop_specific_query_synthesizer
2,how use contextlib closing?,[[KEYWORDS] closing method contextlib closing\...,you can use contextlib.closing(generator) in t...,Python Software Developer,POOR_GRAMMAR,SHORT,single_hop_specific_query_synthesizer
3,What is the purpose of using contextlib.closin...,[[KEYWORDS] closing method contextlib closing\...,The purpose of using contextlib.closing with g...,System Programmer,WEB_SEARCH_LIKE,MEDIUM,single_hop_specific_query_synthesizer
4,What is the role of the 'From' address in the ...,[[KEYWORDS] envelope prompt input strip split ...,"In the smtplib example, the 'From' address is ...",System Programmer,POOR_GRAMMAR,LONG,single_hop_specific_query_synthesizer


In [73]:
# >>>> Chain 응답들을 저장할 list
response_list = []
# Chain 이 반환한 context들을 지정할 list
retrieved_context_list = []

for user_input in eval_df['user_input']:
    res = chain.invoke(user_input)
    response_list.append(res['response'])
    retrieved_context_list.append(res['retrieved_context'])

# >>>> eval_df에 응답과 context 추가
eval_df['response'] = response_list
eval_df['retrieved_contexts'] = retrieved_context_list
eval_df.head()

,user_input,reference_contexts,reference,persona_name,query_style,query_length,synthesizer_name,response,retrieved_contexts
0,What does SIGUSR2 represent in signal handling?,[[KEYWORDS] signals\n[TITLE] signal\n[H1] sign...,SIGUSR2 is defined as User-defined signal 2.,Email Automation Developer,WEB_SEARCH_LIKE,SHORT,single_hop_specific_query_synthesizer,[1] 핵심 개념 요약 \n- `SIGUSR2`는 사용자 정의 신호 2(User-...,[[KEYWORDS] signals\n[TITLE] signal\n[H1] sign...
1,What does SIGWINCH represent in signal handling?,[[KEYWORDS] signals\n[TITLE] signal\n[H1] sign...,SIGWINCH represents the window resize signal.,Python Software Developer,PERFECT_GRAMMAR,SHORT,single_hop_specific_query_synthesizer,"[1] 핵심 개념 요약 \n- `SIGWINCH`는 ""Window size cha...",[[KEYWORDS] signals\n[TITLE] signal\n[H1] sign...
2,how use contextlib closing?,[[KEYWORDS] closing method contextlib closing\...,you can use contextlib.closing(generator) in t...,Python Software Developer,POOR_GRAMMAR,SHORT,single_hop_specific_query_synthesizer,[1] 핵심 개념 요약 \n- `contextlib`의 `closing()` 함수...,[[KEYWORDS] closing close method thing close u...
3,What is the purpose of using contextlib.closin...,[[KEYWORDS] closing method contextlib closing\...,The purpose of using contextlib.closing with g...,System Programmer,WEB_SEARCH_LIKE,MEDIUM,single_hop_specific_query_synthesizer,[1] 핵심 개념 요약 \n`contextlib.closing`은 Python에서...,[[KEYWORDS] closing method contextlib closing\...
4,What is the role of the 'From' address in the ...,[[KEYWORDS] envelope prompt input strip split ...,"In the smtplib example, the 'From' address is ...",System Programmer,POOR_GRAMMAR,LONG,single_hop_specific_query_synthesizer,[1] 핵심 개념 요약 \n- 'From' 주소는 이메일을 보내는 주체를 표시하는...,[[KEYWORDS] envelope prompt input strip split ...


In [74]:
from ragas import EvaluationDataset
# from_xxxx() xxxx 타입의 객체를 EvaluationDataset객체로 변환.
eval_dataset = EvaluationDataset.from_pandas(
    eval_df[["user_input", "retrieved_contexts", "response", "reference"]]
)
eval_dataset

EvaluationDataset(features=['user_input', 'retrieved_contexts', 'response', 'reference'], len=8)

In [75]:
eval_llm = LangchainLLMWrapper(ChatOpenAI(model=OPENAI_MODEL))
eval_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model=EMBEDDING_MODEL))
# 평가할 함수들을 List로 묶어준다.
metrics = [
    LLMContextRecall(llm=eval_llm),
    LLMContextPrecisionWithReference(llm=eval_llm),
    Faithfulness(llm=eval_llm),
    AnswerRelevancy(llm=eval_llm, embeddings=eval_embeddings)
]

# Run Evaluation
eval_results = evaluate(dataset=eval_dataset, metrics=metrics)

# Convert to Pandas DataFrame for easier viewing
df_results = eval_results.to_pandas()
display(df_results)

/var/folders/by/sbz15vxn1xz8499rjvlm_ctm0000gn/T/ipykernel_57529/57438904.py:1: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  eval_llm = LangchainLLMWrapper(ChatOpenAI(model=OPENAI_MODEL))
/var/folders/by/sbz15vxn1xz8499rjvlm_ctm0000gn/T/ipykernel_57529/57438904.py:2: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  eval_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model=EMBEDDING_MODEL))
Evaluating: 100%|██████████| 32/32 [02:09<00:00,  4.03s/it]


,user_input,retrieved_contexts,response,reference,context_recall,llm_context_precision_with_reference,faithfulness,answer_relevancy
0,What does SIGUSR2 represent in signal handling?,[[KEYWORDS] signals\n[TITLE] signal\n[H1] sign...,[1] 핵심 개념 요약 \n- `SIGUSR2`는 사용자 정의 신호 2(User-...,SIGUSR2 is defined as User-defined signal 2.,1.0,1.000000,0.285714,0.859423
1,What does SIGWINCH represent in signal handling?,[[KEYWORDS] signals\n[TITLE] signal\n[H1] sign...,"[1] 핵심 개념 요약 \n- `SIGWINCH`는 ""Window size cha...",SIGWINCH represents the window resize signal.,1.0,1.000000,0.928571,0.748584
2,how use contextlib closing?,[[KEYWORDS] closing close method thing close u...,[1] 핵심 개념 요약 \n- `contextlib`의 `closing()` 함수...,you can use contextlib.closing(generator) in t...,1.0,0.866667,0.866667,0.797925
3,What is the purpose of using contextlib.closin...,[[KEYWORDS] closing method contextlib closing\...,[1] 핵심 개념 요약 \n`contextlib.closing`은 Python에서...,The purpose of using contextlib.closing with g...,1.0,1.000000,1.000000,0.836581
4,What is the role of the 'From' address in the ...,[[KEYWORDS] envelope prompt input strip split ...,[1] 핵심 개념 요약 \n- 'From' 주소는 이메일을 보내는 주체를 표시하는...,"In the smtplib example, the 'From' address is ...",1.0,1.000000,0.800000,0.685060
5,What is the purpose of the 'From' address in t...,[[KEYWORDS] envelope prompt input strip split ...,[1] 핵심 개념 요약 \n- 'From' 주소는 이메일 메시지의 발신자를 지정하...,The 'From' address in the smtplib example is u...,1.0,1.000000,0.800000,0.698187
6,What is the purpose of the Expr class in the a...,[[KEYWORDS] Constant Expr\n[TITLE] ast\n[H1] a...,[1] 핵심 개념 요약 \n`Expr` 클래스는 Python의 추상 구문 트리(A...,The Expr class in the ast module is used to re...,0.5,0.638889,0.857143,0.856180
7,wat is the match_case in python?,[[KEYWORDS] compare_caseless\n[TITLE] unicode\...,[1] 핵심 개념 요약 \n- `match_case`는 Python의 패턴 매칭 ...,The match_case is a construct used in Python's...,1.0,0.679167,1.000000,0.796042


- **Retrieval**
    -  질문에 대해 검색한 문서(context)들에 대한 평가
    -  **Context Precision(문맥 정밀도)**
        -  검색된 문서(context)들 중 질문과 관련 있는 것들이 **얼마나 상위 순위에 위치하는지** 평가하는 지표.
    -  **Context Recall(문맥 재현률)**
        -  검색된 문서(context)가 정답(ground-truth)의 정보를 얼마나 포함하고 있는지 평가하는 지표.
- **Generation**
    - llm 모델이 생성한 답변에 대한 평가 지표들.
    - **Faithfulness(신뢰성)**
        -  생성된 답변과 검색된 문서(context)간의 관련성을 평가하는 지표
        -  생성된 답변이 주어진 문맥(context)에 얼마나 충실한지를 평가하는 지표로 할루시네이션에 대한 평가로 볼 수있다.
    - **Answer relevancy(답변 적합성)**
        - 생성된 답변과 사용자의 질문간의 관련성을 평가하는 지표
        - 생성된 답변이 사용자의 질문과 얼마나 관련성이 있는지를 평가하는 지표.

In [77]:
print("\n📊 Evaluation Results:")
print(eval_results)


📊 Evaluation Results:
{'context_recall': 0.9375, 'llm_context_precision_with_reference': 0.8981, 'faithfulness': 0.8173, 'answer_relevancy': 0.7847}
